# Л7: Использование Inference API для работы с моделями ИИ (Часть 3). Создание ассистента для написания кода

**Inference API** - это сервис, который обеспечивает быстрый доступ к многочисленным предобученным моделями ИИ, размещенным на инфраструктуре HuggingFace. Используя его нет необходимости в загрузке модели и запуску модели локально. Таким образом можно создавать прототипы приложений для решения различных задач не переживая о вычислительных мощностях своего компьютера. 

**Inference API** предоставляет бесплатный и мгновенный доступ к популярным и эффективным моделями для решения следующего спектра задача:
* Генерация текста: включает большие языковые модели и подсказки для вызова инструментов, генерируйте и экспериментируйте с высококачественными ответами.
* Генерация изображений: легко создавайте персонализированные изображения.
* Работа с документами.
* Классические задачи ИИ: готовые к использованию модели для классификации текста, изображений, распознавания речи и многого другого.


**В этой работе мы познакомимся с этим инструментом и в качестве примера создадим собственного умного ассистента для написания программного кода.**

## 1/ Подготовка среды выполнения
На данной этапе вам наобходимо подготовить виртуальное окружение и установить все необходимые библиотеки.

1. Создать и активировать (или только активировать, если ранне создавали) виртуальной окружение `python`.

В терминале вводим следующие команды команды:

*создаем виртуальное окружение с помощью `python-venv`*
```
python -m venv env
```
*активируем виртуальное окружение*
```
env\Scripts\activate
```
**Примечание.** `env` - это название вашего виртуального окружения, назвать его можете как угодно.

После этого можем выбрать наш локальный интерпрететор pyhton, нажав на кнопку выше "Select kernel".

2. Устанавливаем все необходимые библиотеки

**Примечание.** Библиотеки установятся внутрь вашего виртуального окружения.

Нам понадобятся библиотеки Diffusers, Transformers, Accelerate.

```
pip install transformers
```
Также для работы вышеперечисленных бибилиотек потребуется PyTorch:
```
pip install torch
```
Библиотека Gradio для создания web-приложения.
```
pip install gradio
```
Также для работы с изображение нам потребуется библиотека PIL (Python Image Library) `Pillow`
```
pip install Pillow
```

## 2/ Начало рабты. Создание Inference Client

Для бессерверного обращение к модели (то есть без запуска ее на своем каком-то сервере или локально) необходимо отправить запрос, используя Inference API. В результате этого запроса мы получим ответ - Inference (то есть вывод модели). Запрос можно формировать разынми способами, но в этой работе предлагается использовать Python библиотеку `huggingface_hub`. Выполните установку
```
pip install --upgrade huggingface_hub
```

Эта библиотека предоставляет модуль `InferenceClient`, с помощью которого создадим клиент, который будет формировать запрос на сервера Hugging Face. Импортируем этот модуль. Также импортируете Gradio.

In [ ]:
from huggingface_hub import InferenceClient
import gradio as gr

Есть одно НО! Для использования Serverless Inference API необходимо иметь **токен доступа**. Чтобы его получить необходимо зарегистрироваться на сайте Hugging Face, затем перейдите на [странице создания токенов](https://huggingface.co/settings/tokens/new?globalPermissions=inference.serverless.write&tokenType=fineGrained). Создайте `fine-grained` токен с областью действия `Make calls to the serverless Inference API`.

***Примечание***. Не распространяйте свой токен в публичных местах, иначе он будет скопрометирован и Hugging Face его удалит! Токен можно скопировать только один раз в моменте его создания. Сохраните его где-нибудь лично у себя и используете только в коде своего приложения когда необходимо.

Вставьте сюда сгенерированный токен.

In [ ]:
hf_token = "hf_*******************"

Для создания умного ассистента, который умеет писать код лучше нас, будем использовать модель генерации текста `meta-llama/Meta-Llama-3-8B-Instruct`

In [ ]:
ai_model = "meta-llama/Meta-Llama-3-8B-Instruct"

Создадим клиента через класс `InferenceClient`, указав ему используемую модель и свой токен доступа:

In [ ]:
client = InferenceClient(model=ai_model, token=hf_token)

## 3/ Тестирование работы модели в режиме генерации текста (`Text Generation`)
Здесь все просто: пишем запрос к ИИ (промт), в котором грамотно, четко и подробно излагаем наши требования. Например, попросим создать приложение, выводящее на графике сумму трех синусоид в реальном времени:

In [ ]:
prompt = "write one example python code to output a graph of the sum of three sinusoids"

Вызовем у нашего клиента `client` метод `text_generation()` с нашим запросом:

In [ ]:
ans = client.text_generation(prompt=prompt, max_new_tokens=1000)

Сохарним вывод модели в файл для дальнейшего прочтения

In [ ]:
with open('ans.txt', 'w+', encoding='utf-8') as file:
    file.write(ans)

Откройте файл в текстовом редакторе (можно тут). Вы увидеть текст в разметке **Markdown**. Согласно этой разметке, код помещается между специальными символами "```". Скопируйте этот код в блок кода ниже и запустите. Должен сработать...

In [ ]:
# TODO: вставьте сюда код, который придумала модель ИИ
# возможно потребуется установить дополнительные библиотеки для работы кода, посмотрите внимательно на импорты

Скорее всего код у вас заработал! 👩‍💻 

## 4/ ЗАДАНИЕ ✅

Требуется создать приложение на основе Gradio, которое с помощью моделии языковой модели `meta-llama/Meta-Llama-3-8B-Instruct` генерирует программный код и выдает его в виде файла с соответствующи расширением.

Требования к приложению:

- **ВХОДНЫЕ ДАННЫЕ:** текст запроса
- **ВЫХОДНЫЕ ДАННЫЕ:** программный код, файл программного кода.
  - **ЯП:** по умолчанию просим герерировать Python-код, но вы неограничены в своих предпочтениях
- **ГРАФИЧЕСКИЙ ИНТЕРФЕЙС:**
  - текстовое поле ввода запроса
  - текстовое поле вывода кода (с поддержкой подсветки синтаксиса)
  - кнопка выгрузки файла

### 🔎 *Примечание*
Ответ ИИ помимо кода содержит дополнительный поясняющий текст и т.д., **от которого нужно избавится!** Здесь вам нужно из всего ответа извлечь только код, котрый находится между специальными символами. Это можно сделать с помощью стандартных средств python работы со строками. Пользуйтесь google, документацией, другим ИИ, чем угодно для самостоятельного решения этой задачи

## Форма отчетности
В качестве отчета по лабораторной работе вам необходимо предоставить:
1. Файл с кодом вашего приложения (Python Script или Jupyter Nonebook)
2. Скриншот работы графического web-интерфейса вашего приложения

Отчет разместить на [Google диске](https://drive.google.com/drive/folders/1xhGVF935teSij4CIjEqy2XFADXUUBYRc?usp=sharing). 

Найдите папку вашей подгруппы, в ней создайте папку с вашей фамилией. Именно там вы будете помещать все отчеты по лабораторным занятиям. Для отчета по данной работе создайте папку с именем "ЛР#" и поместите туда ваш отчет.